In [21]:
# 関数の準備

import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import scipy.stats as stats

current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, '../conf/'))
sys.path.append(os.path.join(current_dir, '../crawl/'))
sys.path.append(os.path.join(current_dir, '../data_preparing/'))

# my module
import loader
import boatrace_crawler_conf

def make_label_boolean_ver1(for_analysis_df, column_list_label):
    """
    dfのラベルを下記のように変換する。
    1枠の選手に関しては、1着なら1, それ以外なら0, 2枠以降の選手に関しては、3着以内なら1, 3着以外なら0のboolean

    :param for_analysis_df:
    :param column_list_label:
    :return:
    """

    for column_name in column_list_label:
        if column_name == "rank_1":
            # 1枠の着順のカラムに関しては、トップなら1, そうでなければ0のbooleanのカラムにする
            for_analysis_df.loc[for_analysis_df[column_name] != 1, column_name] = 0
        else:
            for_analysis_df.loc[for_analysis_df[column_name] < 3.5, column_name] = 1
            for_analysis_df.loc[for_analysis_df[column_name] > 3.5, column_name] = 0

    return for_analysis_df


def knn(k, train_data, test_data, label_size):
    
    """
    Knnを行う
    :param k:
    :param train_data:
    :param test_data:
    :param label_size:
    :return:
    """
    labels = []

    for test in test_data:
        # すべてのトレインデータとtest（このループステップでラベルを予測したいデータ）との距離を計算したリストを作る
        distances = np.sum((train_data[:, :-(label_size)] - test[:-(label_size)]) ** 2, axis=1)

        # 距離リストの値が小さい順に並べた、トレインデータのインデックスを持つリストを作る
        sorted_train_indexes = np.argsort(distances)

        # インデックスリストを元に、testから近いk個のトレインデータのラベルを取り出す
        sorted_k_labels = train_data[sorted_train_indexes, -(label_size):][:k]

        # sorted_k_labelsの中で最も数の多かったlabelを取り出す
        mode_result = stats.mode(sorted_k_labels)

        # 最も多かったラベルをnumpyx.ndarray形式で取得
        label = mode_result.mode[0]

        labels.append(label)

    # labelsをarrayに変換
    labels = np.array(labels)

    return labels


In [16]:
# 過去のレース結果をdfとして取得
the_merged_df = loader.make_merged_df()
# print(the_merged_df.dtypes)
# print(the_merged_df[["racerName_3", "aveST_frame3"]])

In [22]:
# 実行

# ----------input-------------
# 解析に使う特徴量カラム
fv_list = []
for i in range(1, 7):
    # 各枠の平均ST
    fv_list.append("aveST_frame{0}".format(i))
    # 連帯率
    fv_list.append("placeRate_frame{0}".format(i))
    # 展示順位
    fv_list.append("exhibitionRank_{0}".format(i))
    # 展示タイム
    fv_list.append("exhibitionTime_{0}".format(i))

# 解析に使うラベルカラム: 今回は一枠が一着になるかどうか？を予測
column_list_label = ["rank_1"]

# データのうち、教師データとして使う割合（残りをテストデータとして用いる）
train_data_ratio = 0.7

# k-NNのk (周囲何個までみるか）
k = 100

# --------------------------------


# 解析用dfを作成
for_analysis_df = the_merged_df[fv_list + column_list_label]
# なぜかdtypeがstrになっちゃうのでintに戻す
for_analysis_df = for_analysis_df.astype(float)
print(for_analysis_df.dtypes)

# ラベルをbooleanに変換し，knn用のdfを作成
for_analysis_df = make_label_boolean_ver1(for_analysis_df, column_list_label)

# 教師データとテストデータに分ける (標準化なしのデータ)
train_size = int(len(for_analysis_df) * train_data_ratio)

train_data = for_analysis_df[:train_size].values
test_data = for_analysis_df[train_size:].values

# knnのinput用にlabelのサイズの変数を作成
label_size = len(column_list_label)

# k-NNの実行
pred_labels = knn(k, train_data, test_data, label_size)

# 正答率を計算
accuracy = np.sum(pred_labels == test_data[:, -(label_size):], axis=0) / len(test_data)
print(accuracy)


aveST_frame1        float64
placeRate_frame1    float64
exhibitionRank_1    float64
exhibitionTime_1    float64
aveST_frame2        float64
placeRate_frame2    float64
exhibitionRank_2    float64
exhibitionTime_2    float64
aveST_frame3        float64
placeRate_frame3    float64
exhibitionRank_3    float64
exhibitionTime_3    float64
aveST_frame4        float64
placeRate_frame4    float64
exhibitionRank_4    float64
exhibitionTime_4    float64
aveST_frame5        float64
placeRate_frame5    float64
exhibitionRank_5    float64
exhibitionTime_5    float64
aveST_frame6        float64
placeRate_frame6    float64
exhibitionRank_6    float64
exhibitionTime_6    float64
rank_1              float64
dtype: object
[0.61812205]
